In [41]:
from PIL import Image, ImageChops, ImageEnhance
import numpy as np
import tensorflow as tf
import keras
import os
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
import zipfile
import shutil
import random
from shutil import copyfile

In [42]:
def convert_to_ela_image(image):
        quality=90;
        #im=image.convert('RGB')
        im = Image.fromarray(image,'RGB')
        im.save('resaved.jpg','JPEG', quality=quality)
        resaved_im=Image.open('resaved.jpg')
        ela_im = ImageChops.difference(im, resaved_im)
        os.remove('resaved.jpg')
        extrema = ela_im.getextrema()
        max_diff = max([ex[1] for ex in extrema])
        if max_diff == 0:
            max_diff = 1
        scale = 255.0/max_diff 

        ela_im = ImageEnhance.Brightness(ela_im).enhance(scale)
        return np.array(ela_im,dtype='float64')

In [43]:
model=tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32,(5,5),padding='valid',activation='relu',input_shape=(128,128,3)),
    tf.keras.layers.Conv2D(32,(5,5),strides=(2,2),padding='valid',activation='relu'),
    tf.keras.layers.MaxPool2D((2,2),strides=None,padding='valid',data_format='channels_last'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2,activation='softmax')
]);
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 124, 124, 32)      2432      
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 60, 60, 32)        25632     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 30, 30, 32)        0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 30, 30, 32)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 28800)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               7373056   
_________________________________________________________________
dropout_28 (Dropout)         (None, 256)             

In [44]:
from tensorflow.keras.optimizers import RMSprop
model.compile(optimizer=RMSprop(lr=0.01,rho=0.9, epsilon=1e-08, decay=0.0),loss='categorical_crossentropy',metrics=['accuracy']);

In [46]:
local_zip='Data.zip';
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('Data')
zip_ref.close()

In [48]:
print(len(os.listdir('Data/data')))

2001


In [49]:
os.mkdir('Dataset');
os.mkdir('Dataset/scale');
os.mkdir('Dataset_1');
os.mkdir('Dataset_1/scale');
os.mkdir('Dataset_1/tamp');

In [50]:
names=os.listdir('Data/data');
for i in range(0,len(names)):
    file=names[i];
    path='Data/data/'+file;
    if(file.find('tamp') != -1):
        shutil.move(path,'Dataset_1/tamp/');
    else:
        shutil.move(path,'Dataset/scale/');

In [51]:
import random
FILE_PATH = "Dataset/scale"
TARGET_NORMAL_DIR = "Dataset_1/scale"

In [52]:
image_names = os.listdir(FILE_PATH)
random.shuffle(image_names)

In [53]:
for i in range (700):
    
    image_name = image_names[i]
    image_path = os.path.join(FILE_PATH,image_name)
    
    target_path = os.path.join(TARGET_NORMAL_DIR,image_name)
    shutil.copy2(image_path,target_path)
    
    #print ("copying image ",i)

In [54]:
print(len(os.listdir('Dataset_1/tamp')))
print(len(os.listdir('Dataset_1/scale')))

700
700


In [55]:
os.mkdir('Image_Dataset');
os.mkdir('Image_Dataset/Train');
os.mkdir('Image_Dataset/Test');
os.mkdir('Image_Dataset/Train/tamp');
os.mkdir('Image_Dataset/Train/scale');
os.mkdir('Image_Dataset/Test/tamp');
os.mkdir('Image_Dataset/Test/scale');

In [56]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    listing=os.listdir(SOURCE);
    random.sample(listing,len(listing));
    list2=listing[0:int(len(listing)-SPLIT_SIZE*len(listing))];
    list3=list(set(listing)-set(list2));
    c=0;
    for i in list2:
        if(os.path.getsize(SOURCE+i)>0):
            copyfile(SOURCE+i,TRAINING+i);
    for i in list3:
        if(os.path.getsize(SOURCE+i)>0):
            copyfile(SOURCE+i,TESTING+i);

tamp_SOURCE_DIR = 'Dataset_1/tamp/'
TRAINING_tamp_DIR = 'Image_Dataset/Train/tamp/'
TESTING_tamp_DIR = 'Image_Dataset/Test/tamp/'
scale_SOURCE_DIR = 'Dataset_1/scale/'
TRAINING_scale_DIR = 'Image_Dataset/Train/scale/'
TESTING_scale_DIR = 'Image_Dataset/Test/scale/'
split_size = .2
split_data(tamp_SOURCE_DIR, TRAINING_tamp_DIR, TESTING_tamp_DIR, split_size)
split_data(scale_SOURCE_DIR, TRAINING_scale_DIR, TESTING_scale_DIR, split_size)

In [57]:
print(len(os.listdir('Image_Dataset/Train/scale')))
print(len(os.listdir('Image_Dataset/Test/scale')))

560
140


In [58]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
TRAINING_DIR = 'Image_Dataset/Train'
train_datagen = ImageDataGenerator(
      preprocessing_function=convert_to_ela_image,
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,target_size=(128,128),batch_size=5,class_mode='categorical')
VALIDATION_DIR = 'Image_Dataset/Test'
validation_datagen = ImageDataGenerator(rescale=1.0/255.0);
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,target_size=(128,128),batch_size=10,class_mode='categorical');

Found 1119 images belonging to 2 classes.
Found 280 images belonging to 2 classes.


In [60]:
history = model.fit(
      train_generator,
      steps_per_epoch=280,  # 2000 images = batch_size * steps
      epochs=10,
      validation_data=validation_generator,
      validation_steps=20)

Epoch 1/10
 51/280 [====>.........................] - ETA: 9:17 - loss: 0.6958 - accuracy: 0.4931

UnknownError:  PermissionError: [WinError 5] Access is denied: 'resaved.jpg'
Traceback (most recent call last):

  File "C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\script_ops.py", line 243, in __call__
    ret = func(*args)

  File "C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\autograph\impl\api.py", line 309, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 785, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 801, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\user\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\keras\engine\data_adapter.py", line 932, in generator_fn
    yield x[i]

  File "C:\Users\user\anac